<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-14 15:42:40
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.33 C
-------------------
Today PnL: -24.70 K (-0.19%)
Current PnL: -22.06 L (-15.01%)
CY Booked + Current PnL: -13.79 L (-9.38%)
-------------------
Total profit:  2.61 L
Total loss:  -24.67 L
-------------------
Total Booked + Current PnL: 13.19 L (10.98%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.23%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.54 L (61.39%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBIN,760.30,863.00,-11.34,M-LC,3.83,219036.0,17556.0,9659.0,0.57,8.71,4.41,13.51,61.0,1.82,1.68,22.41,XY25,NTT,BANKS
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.22,111240.0,6928.0,9722.0,-0.75,6.64,8.74,15.96,11.0,0.71,0.85,24.66,X5K,ATH,METALS
4,APOLLOHOSP,7098.95,8285.00,-13.86,H-LC,5.94,172073.0,15896.0,10204.0,0.17,10.18,5.93,16.71,19.0,1.56,1.32,29.23,X40N,BTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,11.67,H-MC,13.83,159396.0,33196.0,12975.0,1.36,26.30,8.14,36.59,111.0,2.56,1.22,42.11,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-23.74,H-SC,10.26,227650.0,34630.0,13409.0,1.59,17.94,5.89,24.89,134.0,2.58,1.75,45.48,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJOOENG,102.50,143.33,-32.41,H-SC,4.59,102560.0,60.0,40768.0,1.54,0.06,39.75,39.83,116.0,0.00,0.79,1.54,AR,ATH,MISC
25,DABUR,505.20,735.00,-10.32,H-MC,3.46,195351.0,-1677.0,91307.0,-0.52,-0.85,46.74,45.49,103.0,-0.02,1.50,12.08,XY24,BTT,FMCG
90,VOLTAS,1278.28,1918.49,-8.91,H-MC,7.71,191220.0,-522.0,96547.0,-0.10,-0.27,50.49,50.08,101.0,-0.01,1.47,6.51,XY25,ATH,AC
58,PGHH,13388.00,18062.58,-31.21,H-MC,6.36,199020.0,-1800.0,71926.0,1.28,-0.90,36.14,34.92,82.0,-0.03,1.53,3.74,X40,ATH,FMCG


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-19.78,H-LC,2.53,214268.0,9768.0,31647.0,0.19,4.78,14.77,20.25,10.0,0.31,1.64,13.93,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-11.88,H-LC,3.20,171952.0,-29434.0,117048.0,1.14,-14.62,68.07,43.51,27.0,-0.25,1.32,19.70,X40,BTT,PAINTS
57,NESTLEIND,2268.60,2755.0,-17.13,H-LC,3.24,127460.0,-137966.0,194874.0,-0.68,-51.98,152.89,21.44,8.0,-0.71,0.98,2.42,XY25,NTT,FMCG
53,LTIM,5564.16,7230.2,-7.23,H-LC,3.77,183888.0,-16422.0,76405.0,-0.13,-8.20,41.55,29.94,17.0,-0.21,1.41,27.04,X200,ATH,IT
37,HINDUNILVR,2413.81,2723.0,-11.65,H-LC,5.77,277827.0,7480.0,27144.0,-0.58,2.77,9.77,12.81,28.0,0.28,2.13,16.19,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,VALIANTORG,512.64,838.0,-325.20,H-SC,16.79,95823.0,-37463.0,122059.0,5.00,-28.11,127.38,63.47,143.0,-0.31,0.74,60.62,XR,NTT,CHEMICALS
10,BAJAJHFL,122.26,152.0,-12.36,H-MC,5.32,184861.0,-15645.0,64424.0,3.83,-7.80,34.85,24.33,98.0,-0.24,1.42,6.60,X40N,BTT,FINANCE
29,EASEMYTRIP,18.12,26.4,-6.88,M-SC,24.78,89676.0,-87139.0,167936.0,3.72,-49.28,187.27,45.70,197.0,-0.52,0.69,4.31,XY24,NTT,TRAVEL
92,WIPRO,243.46,420.0,-12.89,M-LC,7.33,153022.0,2077.0,107376.0,2.14,1.38,70.17,72.51,54.0,0.02,1.17,7.48,XR,NTT,IT
30,GREENPANEL,375.16,537.0,198.45,M-SC,5.71,133297.0,-49781.0,128765.0,1.71,-27.19,96.60,43.14,230.0,-0.39,1.02,25.38,XY24,NTT,MISC


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,1550.24,2096.0,-41.72,M-SC,7.19,87125.0,-41545.0,86846.0,-4.30,-32.29,99.68,35.20,209.0,-0.48,0.67,0.00,X40,NTT,FOOTWEAR
65,SAIL,130.64,228.0,31.52,M-MC,10.18,207277.0,-17685.0,185347.0,-2.97,-7.86,89.42,74.53,193.0,-0.10,1.59,20.35,XY24,BTT,STEEL
18,CAMPUS,294.86,393.0,-30.83,M-SC,2.16,144185.0,-23590.0,79432.0,-2.80,-14.06,55.09,33.28,208.0,-0.30,1.11,11.63,XY24,NTT,FOOTWEAR
70,SHALBY,261.39,327.0,935.09,M-SC,6.89,140818.0,-40325.0,85800.0,-2.48,-22.26,60.93,25.10,232.0,-0.47,1.08,12.85,XY24,NTT,HEALTHCARE
84,UJJIVANSFB,52.77,60.0,78.53,H-SC,20.40,112509.0,-29970.0,49493.0,-2.46,-21.03,43.99,13.70,163.0,-0.61,0.86,32.79,OX40N,NTT,BANKS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,92.10,M-SC,10.35,82753.0,-18024.0,18115.0,-0.53,-17.89,21.89,0.09,243.0,-0.99,0.64,8.72,OX40N,NTT,DURABLES
20,CERA,7989.07,9475.0,-19.33,H-SC,3.12,115164.0,-28639.0,55382.0,-0.19,-19.92,48.09,18.60,148.0,-0.52,0.88,27.09,OX40N,NTT,CERAMICS
84,UJJIVANSFB,52.77,60.0,78.53,H-SC,20.40,112509.0,-29970.0,49493.0,-2.46,-21.03,43.99,13.70,163.0,-0.61,0.86,32.79,OX40N,NTT,BANKS
43,INDIGOPNTS,1407.73,1408.0,109.26,M-SC,2.91,137293.0,-37266.0,37303.0,0.73,-21.35,27.17,0.02,221.0,-1.00,1.05,19.05,OX40N,NTT,PAINTS
49,KANSAINER,299.63,340.0,-69.25,H-SC,2.82,211905.0,-57762.0,94086.0,-0.44,-21.42,44.40,13.47,142.0,-0.61,1.63,7.18,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.69,H-LC,15.98,235739.0,-60195.0,154103.0,-0.46,-20.34,65.37,31.73,2.0,-0.39,1.81,0.64,X40,BTT,IT
45,INFY,1461.46,2275.00,-20.67,H-LC,10.93,259138.0,-2463.0,148097.0,1.48,-0.94,57.15,55.67,4.0,-0.02,1.99,5.11,X40,BTT,IT
89,VBL,492.64,671.64,-11.16,H-LC,10.14,269995.0,4955.0,91339.0,-1.81,1.87,33.83,36.33,5.0,0.05,2.07,15.42,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.20,H-LC,10.47,196170.0,-20349.0,113249.0,-1.12,-9.40,57.73,42.91,7.0,-0.18,1.51,2.73,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-17.13,H-LC,3.24,127460.0,-137966.0,194874.0,-0.68,-51.98,152.89,21.44,8.0,-0.71,0.98,2.42,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,11.67,H-MC,13.83,159396.0,33196.0,12975.0,1.36,26.3,8.14,36.59,111.0,2.56,1.22,42.11,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,11.67,H-MC,13.83,159396.0,33196.0,12975.0,1.36,26.30,8.14,36.59,111.0,2.56,1.22,42.11,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,7.49,H-SC,12.20,128180.0,2883.0,56117.0,-1.32,2.30,43.78,47.09,141.0,0.05,0.98,60.43,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,3.35,H-MC,6.93,178520.0,13850.0,54109.0,1.67,8.41,30.31,41.28,91.0,0.26,1.37,17.06,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-4.72,H-SC,3.87,108321.0,6317.0,37609.0,1.24,6.19,34.72,43.06,123.0,0.17,0.83,24.37,X40N,ATH,MISC
35,HEROMOTOCO,4311.81,5949.07,-11.08,H-MC,7.83,164784.0,13871.0,43437.0,-1.27,9.19,26.36,37.97,92.0,0.32,1.26,36.82,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-11.08,H-MC,7.83,164784.0,13871.0,43437.0,-1.27,9.19,26.36,37.97,92.0,0.32,1.26,36.82,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.65,H-SC,11.98,134927.0,11591.0,122176.0,-0.28,9.40,90.55,108.45,119.0,0.09,1.04,33.85,AR,ATH,MISC
50,KPIGREEN,497.21,731.35,7.49,H-SC,12.20,128180.0,2883.0,56117.0,-1.32,2.30,43.78,47.09,141.0,0.05,0.98,60.43,MH,ATH,POWER
92,WIPRO,243.46,420.00,-12.89,M-LC,7.33,153022.0,2077.0,107376.0,2.14,1.38,70.17,72.51,54.0,0.02,1.17,7.48,XR,NTT,IT
91,WHIRLPOOL,1219.98,2270.00,-44.96,M-SC,4.28,93052.0,1554.0,77196.0,-0.05,1.70,82.96,86.07,219.0,0.02,0.71,34.61,XR,NTT,DURABLES


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.58,72883.0,-40666.0,80718.0,-0.37,-35.81,110.75,35.27,268.0,-0.50,0.56,78.38,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-4.17,H-SC,15.58,89152.0,-11811.0,106341.0,-0.94,-11.70,119.28,93.63,149.0,-0.11,0.68,32.79,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.51,H-SC,13.78,90150.0,-7830.0,29704.0,-2.36,-7.99,32.95,22.32,152.0,-0.26,0.69,30.75,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-325.20,H-SC,16.79,95823.0,-37463.0,122059.0,5.00,-28.11,127.38,63.47,143.0,-0.31,0.74,60.62,XR,NTT,CHEMICALS
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.22,111240.0,6928.0,9722.0,-0.75,6.64,8.74,15.96,11.0,0.71,0.85,24.66,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.00,-23.74,H-SC,10.26,227650.0,34630.0,13409.0,1.59,17.94,5.89,24.89,134.0,2.58,1.75,45.48,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-11.08,H-MC,7.83,164784.0,13871.0,43437.0,-1.27,9.19,26.36,37.97,92.0,0.32,1.26,36.82,AR,ATH,AUTO
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.58,72883.0,-40666.0,80718.0,-0.37,-35.81,110.75,35.27,268.0,-0.50,0.56,78.38,XR,NTT,HOTELS
70,SHALBY,261.39,327.00,935.09,M-SC,6.89,140818.0,-40325.0,85800.0,-2.48,-22.26,60.93,25.10,232.0,-0.47,1.08,12.85,XY24,NTT,HEALTHCARE
89,VBL,492.64,671.64,-11.16,H-LC,10.14,269995.0,4955.0,91339.0,-1.81,1.87,33.83,36.33,5.0,0.05,2.07,15.42,X40N,ATH,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.47
1,25,41.17
2,50,70.98


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.34
LC    34.35
MC    26.31
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.81
X40N     15.65
X40      13.37
XY25     10.65
XR        9.07
AR        8.44
OX40N     7.47
X5K       2.27
MH        1.74
X200      1.41
SR        1.12
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.96
H-SC    24.53
H-MC    23.38
M-SC    13.32
M-LC     7.30
M-MC     2.59
L-SC     1.49
L-LC     1.09
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.19,-11.53,47.39
IT,12.00,-19.24,78.79
FINANCE,8.24,-21.36,66.25
BANKS,7.62,-14.88,63.66
MISC,5.92,-19.26,87.63
PAINTS,5.66,-18.41,41.75
HEALTHCARE,5.37,6.20,17.49
ELECTRICAL,4.74,-13.77,63.04
AUTO,4.49,-15.43,65.66


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2884117.0,23
XR,1112868.0,13
AR,1111271.0,10
X40,829981.0,10
XY25,661340.0,8
X40N,567430.0,12
OX40N,484102.0,10
SR,196795.0,2
X5K,141771.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2820624.0,28
M-SC,1510284.0,17
H-LC,1354095.0,18
H-MC,1331111.0,18
M-LC,426449.0,5
M-MC,340353.0,2
L-SC,267786.0,3
L-MC,58724.0,1
L-LC,44495.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      949269.0      6
M-SC       XY24      876884.0      7
H-SC       AR        673461.0      5
           XR        524771.0      6
H-LC       X40       515464.0      5
H-MC       XY24      498535.0      5
M-MC       XY24      340353.0      2
H-LC       XY25      260177.0      3
           AR        233267.0      2
H-MC       X40       227671.0      4
           XY25      222371.0      2
H-LC       X40N      217349.0      5
H-SC       OX40N     210657.0      4
           X40N      209554.0      4
           SR        196795.0      2
M-LC       XY24      177365.0      2
L-SC       XR        177194.0      2
M-SC       AR        161106.0      2
H-MC       X40N      140527.0      3
M-SC       OX40N     134142.0      4
M-LC       X5K       132049.0      1
M-SC       XR        126668.0      2
           XY25      124638.0      1
H-MC       XR        118135.0      1
M-LC       XR        107376.0      1
L-SC       OX40N      90592.0      1
M-SC       X40        86846.0      1
H-LC       X200       76405.0      1
L-MC       XR         58724.0      1
H-SC       MH         56117.0      1
H-MC       OX40N      48711.0      1
L-LC       XY25       44495.0      1
H-MC       AR         43437.0      1
H-LC       XY24       41711.0      1
H-MC       MH         31724.0      1
H-LC       X5K         9722.0      1
M-LC       XY25        9659.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
